## RDP Learning

In this notebook, we show how to perform PAC learning
over RDPs.

### Experiment 1: Rotating MAB

In [16]:
from functools import partial
from typing import Tuple, Set, Dict

import gym
import numpy as np
from gym.wrappers import TimeLimit
from notebooks.utils import render_digraph

from src import NonMarkovianRotatingMAB
from src.learn_pdfa.base import learn_pdfa, Algorithm
from src.learn_pdfa.common import MultiprocessedGenerator
from src.learn_rdps import random_exploration_policy, RDPGenerator
from src.pdfa.render import to_graphviz_from_graph
from src.pdfa.types import Character

env = NonMarkovianRotatingMAB(winning_probs=[0.9, 0.2])

print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

s = env.reset()
print(f"Initial state: {s}")

action = 0
sp, reward, _, _ = env.step(action)
print("-" * 10)
print(f"Action: {action}")
print(f"Next state: {sp}")
print(f"Reward: {reward}")

sp, reward, _, _ = env.step(action)
print("-" * 10)
print(f"Action: {action}")
print(f"Next state: {sp}")
print(f"Reward: {reward}")


Observation space: Discrete(2)
Action space: Discrete(2)
Initial state: 0
----------
Action: 0
Next state: 1
Reward: 1.0
----------
Action: 0
Next state: 1
Reward: 1.0


### Learning

In [17]:
def learning_rotating_mab(
    stop_probability: float,
    winning_probabilities: Tuple[float, ...],
    max_episode_steps: int,
    nb_samples: int,
    delta: float,
    n_upperbound: int,
    nb_processes: int = 8,
) -> Tuple[RDPGenerator, Tuple[Set[int], Dict[int, Dict[Character, int]]]]:
    """Test learning of Rotating MAB."""
    env = gym.make("NonMarkovianRotatingMAB-v0", winning_probs=winning_probabilities)
    env = TimeLimit(env, max_episode_steps=max_episode_steps)

    policy = partial(random_exploration_policy, env)

    rdp_generator = RDPGenerator(
        env, policy=policy, nb_rewards=2, stop_probability=stop_probability
    )

    examples = rdp_generator.sample(n=1000)

    print(
        f"Apriori expected length of traces: 1/stop_prob = {1 / (stop_probability + np.finfo(float).eps)}"
    )
    print(f"Average length of traces: {np.mean([len(e) for e in examples])}")

    mp_rdp_generator = MultiprocessedGenerator(rdp_generator, nb_processes=nb_processes)
    v, t = learn_pdfa(
        algorithm=Algorithm.BALLE,
        nb_samples=nb_samples,
        sample_generator=mp_rdp_generator,
        alphabet_size=rdp_generator.alphabet_size(),
        delta=delta,
        n=n_upperbound,
    )
    return rdp_generator, (v, t)

In [18]:
rdp_generator, (v, t) = learning_rotating_mab(
    stop_probability=0.2,
    winning_probabilities=(0.7, 0.3),
    max_episode_steps=15,
    nb_samples=100000,
    delta=0.1,
    n_upperbound=4,
    nb_processes=8)

render_digraph(to_graphviz_from_graph(v, t, char2str=lambda c: str(rdp_generator.decoder(c))))


Apriori expected length of traces: 1/stop_prob = 4.999999999999994
Average length of traces: 3.679


[2020-11-06 18:56:41,849][src.learn_pdfa][INFO] Parameters: ('BalleParams(sample_generator=<src.learn_pdfa.common.MultiprocessedGenerator '
 'object at 0x7fce23e3bf50>, nb_samples=100000, n=4, alphabet_size=8, '
 'delta=0.1)')
[2020-11-06 18:56:46,813][src.learn_pdfa][INFO] Iteration 0
[2020-11-06 18:56:47,510][src.learn_pdfa][INFO] Iteration 1
[2020-11-06 18:56:48,198][src.learn_pdfa][INFO] Iteration 2
[2020-11-06 18:56:48,820][src.learn_pdfa][INFO] Iteration 3
[2020-11-06 18:56:49,467][src.learn_pdfa][INFO] Iteration 4
[2020-11-06 18:56:50,129][src.learn_pdfa][INFO] Iteration 5
[2020-11-06 18:56:50,947][src.learn_pdfa][INFO] Iteration 6
[2020-11-06 18:56:51,721][src.learn_pdfa][INFO] Iteration 7
[2020-11-06 18:56:52,602][src.learn_pdfa][INFO] Iteration 8
/home/marcofavorito/.local/share/virtualenvs/PAC-RDPs-code-h40RRNYH/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 'shape' argument should be used instead of 'dims'
  # Remove the CWD from sys.path while we

In [19]:
rdp_generator, (v, t) = learning_rotating_mab(
    stop_probability=0.2,
    winning_probabilities=(1.0, 0.0, 0.0),
    max_episode_steps=15,
    nb_samples=500000,
    delta=0.1,
    n_upperbound=5,
    nb_processes=8)

render_digraph(to_graphviz_from_graph(v, t, char2str=lambda c: str(rdp_generator.decoder(c))))

Apriori expected length of traces: 1/stop_prob = 4.999999999999994
Average length of traces: 3.679


[2020-11-06 18:56:56,392][src.learn_pdfa][INFO] Parameters: ('BalleParams(sample_generator=<src.learn_pdfa.common.MultiprocessedGenerator '
 'object at 0x7fce22920150>, nb_samples=500000, n=5, alphabet_size=12, '
 'delta=0.1)')
[2020-11-06 18:57:02,292][src.learn_pdfa][INFO] Iteration 0
[2020-11-06 18:57:02,835][src.learn_pdfa][INFO] Iteration 1
[2020-11-06 18:57:03,367][src.learn_pdfa][INFO] Iteration 2
[2020-11-06 18:57:03,985][src.learn_pdfa][INFO] Iteration 3
[2020-11-06 18:57:04,683][src.learn_pdfa][INFO] Iteration 4
[2020-11-06 18:57:05,395][src.learn_pdfa][INFO] Iteration 5
[2020-11-06 18:57:06,180][src.learn_pdfa][INFO] Iteration 6
[2020-11-06 18:57:06,979][src.learn_pdfa][INFO] Iteration 7
[2020-11-06 18:57:07,780][src.learn_pdfa][INFO] Iteration 8
[2020-11-06 18:57:08,607][src.learn_pdfa][INFO] Iteration 9
/home/marcofavorito/.local/share/virtualenvs/PAC-RDPs-code-h40RRNYH/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 'shape' argument should be use

In [35]:
rdp_generator, (v, t) = learning_rotating_mab(
    stop_probability=0.2,
    winning_probabilities=(0.1, 0.2, 0.9),
    max_episode_steps=100,
    nb_samples=5000000,
    delta=0.05,
    n_upperbound=6,
    nb_processes=8)

render_digraph(to_graphviz_from_graph(v, t, char2str=lambda c: str(rdp_generator.decoder(c))))

Apriori expected length of traces: 1/stop_prob = 4.999999999999994
Average length of traces: 3.84


[2020-11-06 19:15:54,368][src.learn_pdfa][INFO] Parameters: ('BalleParams(sample_generator=<src.learn_pdfa.common.MultiprocessedGenerator '
 'object at 0x7fce3072c490>, nb_samples=5000000, n=6, alphabet_size=12, '
 'delta=0.05)')
[2020-11-06 19:16:01,579][src.learn_pdfa][INFO] Iteration 0
[2020-11-06 19:16:02,968][src.learn_pdfa][INFO] Iteration 1
[2020-11-06 19:16:04,591][src.learn_pdfa][INFO] Iteration 2
[2020-11-06 19:16:06,185][src.learn_pdfa][INFO] Iteration 3
[2020-11-06 19:16:08,592][src.learn_pdfa][INFO] Iteration 4
[2020-11-06 19:16:11,538][src.learn_pdfa][INFO] Iteration 5
[2020-11-06 19:16:13,160][src.learn_pdfa][INFO] Iteration 6
[2020-11-06 19:16:14,856][src.learn_pdfa][INFO] Iteration 7
[2020-11-06 19:16:17,021][src.learn_pdfa][INFO] Iteration 8
[2020-11-06 19:16:19,131][src.learn_pdfa][INFO] Iteration 9
[2020-11-06 19:16:21,708][src.learn_pdfa][INFO] Iteration 10
[2020-11-06 19:16:23,891][src.learn_pdfa][INFO] Iteration 11
[2020-11-06 19:16:28,019][src.learn_pdfa][INFO] 